In [1]:
# imports
from google.colab import drive
import pandas as pd

# mount to drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/F21/CIS550/Final Project/WSJ Articles/'

In [133]:
wsj = pd.read_csv(path + 'oct20-oct21.csv')
stocks_raw = pd.read_csv(path + 'russell1000stocks.csv')

In [202]:
# clean stocks csv
import re

stocks = stocks_raw.copy()
companies_cleaned = stocks_raw['Description']
for idx, company in enumerate(companies_cleaned):
  cleaned = company.replace(',', '')
  cleaned = cleaned.replace('.', '')
  cleaned = cleaned.replace('com', '')
  cleaned = cleaned.replace('Inc', '')
  cleaned = cleaned.replace('Inc', '')
  cleaned = cleaned.replace('Com', '')
  cleaned = cleaned.replace('Corp', '')
  cleaned = re.sub("Class [A-Za-z]", "", cleaned)
  # 'News' is too common word and ticker 'A' too common (ticker/company name detection is not reliable with these companies)
  if cleaned == 'News':
    stocks.drop([idx], inplace=True)
  elif cleaned == 'Agilent Technologies':
    stocks.drop([idx], inplace=True)
  else:
    companies_cleaned[idx] = cleaned.strip()
stocks['Description'] = companies_cleaned
stocks.to_csv(path + 'russell1000stocks_cleaned.csv', index = False)

In [207]:
# create stock ticker/company name to sector mappings
raw_tickers = stocks['Symbol'].values
raw_companies = stocks['Description'].values
tickers = ' '.join([str(elem) for elem in stocks['Symbol']]) # ignore Agilent stock ticker
companies = ' '.join([str(elem) for elem in stocks['Description']])
tickers_to_company = dict(zip(stocks['Symbol'], stocks['Description']))
tickers_to_sector = dict(zip(stocks['Symbol'], stocks['GICS Sector']))
company_to_sector = dict(zip(stocks['Description'], stocks['GICS Sector']))
# print(tickers)
# print(companies)
# print(tickers_to_sector)

In [208]:
# ticker
num_articles = len(wsj)
print("Beginning to add columns to", num_articles, "articles")
title_ticker_mentions = [None] * num_articles
subtitle_ticker_mentions = [None] * num_articles
content_ticker_mentions = [None] * num_articles
# company names
title_company_mentions = [None] * num_articles
subtitle_company_mentions = [None] * num_articles
content_company_mentions = [None] * num_articles
# sector
title_sector_mentions = [None] * num_articles
subtitle_sector_mentions = [None] * num_articles
content_sector_mentions = [None] * num_articles
i = 0
for index, row in wsj.iterrows():
    currTitle = row['Title']
    currSubtitle = row['Subtitle']
    currContent = row['Content']
    # TICKERS
    # title
    title_ticker = set(tickers.split()).intersection(set(currTitle.split()))
    cleaned_title_ticker = []
    if type(title_ticker) == 'set' and title_ticker is not None:
      for tick in title_ticker:
        if tick in raw_tickers:
          cleaned_title_ticker.append(tick)
    if len(cleaned_title_ticker) != 0:
      title_ticker_mentions[index] = cleaned_title_ticker
    # subtitle
    subtitle_ticker = set(tickers.split()).intersection(set(currSubtitle.split()))
    cleaned_subtitle_ticker = []
    if type(subtitle_ticker) == 'set' and subtitle_ticker is not None:
      for tick in subtitle_ticker:
        if tick in raw_tickers:
          cleaned_subtitle_ticker.append(tick)
    if len(cleaned_subtitle_ticker) != 0:
      subtitle_ticker_mentions[index] = cleaned_subtitle_ticker
    # content
    content_ticker = set(tickers.split()).intersection(set(currContent.split()))
    cleaned_content_ticker = []
    if type(content_ticker) == 'set' and content_ticker is not None:
      for tick in content_ticker:
        if tick in raw_tickers:
           cleaned_contentticker.append(tick)
    if len(cleaned_content_ticker) != 0:
      content_ticker_mentions[index] = cleaned_content_ticker
    # COMPANY NAMES
    # title
    title_company = set(companies.split()).intersection(set(currTitle.split()))
    # print("title_company", title_company)
    cleaned_title_company = []
    for title in title_company:
      if title in raw_companies:
        cleaned_title_company.append(title)
    # print("pre", cleaned_title_company)
    if type(title_ticker) == 'set' and title_ticker is not None:
      for tick in cleaned_title_ticker:
        cleaned_title_company.add(tickers_to_company[tick])
    # print("post", cleaned_title_company)
    if len(cleaned_title_company) != 0:
      title_company_mentions[index] = cleaned_title_company
    # subtitle
    subtitle_company = set(companies.split()).intersection(set(currSubtitle.split()))
    cleaned_subtitle_company = []
    for title in subtitle_company:
      if title in raw_companies:
        cleaned_subtitle_company.append(title)
    if type(subtitle_ticker) == 'set' and subtitle_ticker is not None:
      for tick in cleaned_subtitle_ticker:
        cleaned_subtitle_company.add(tickers_to_company[tick])
    if len(cleaned_subtitle_company) != 0:
      subtitle_company_mentions[index] = cleaned_subtitle_company
    # content
    content_company = set(companies.split()).intersection(set(currContent.split()))
    cleaned_content_company = []
    for title in content_company:
      if title in raw_companies:
        cleaned_content_company.append(title)
    if type(content_ticker) == 'set' and content_ticker is not None:
      for tick in cleaned_content_ticker:
        cleaned_content_company.add(tickers_to_company[tick])
    if len(cleaned_content_company) != 0:
      content_company_mentions[index] = cleaned_content_company
    # SECTORS
    # title
    title_sector = []
    title_sector.extend([company_to_sector[x] for x in cleaned_title_company])
    title_sector.extend([tickers_to_sector[x] for x in cleaned_title_ticker])
    if len(title_sector) != 0:
      title_sector_mentions[index] = title_sector
    # subtitle
    subtitle_sector = []
    subtitle_sector.extend([company_to_sector[x] for x in cleaned_subtitle_company])
    subtitle_sector.extend([tickers_to_sector[x] for x in cleaned_subtitle_ticker])
    if len(subtitle_sector) != 0:
      subtitle_sector_mentions[index] = subtitle_sector
    # content
    content_sector = []
    content_sector.extend([company_to_sector[x] for x in cleaned_content_company])
    content_sector.extend([tickers_to_sector[x] for x in cleaned_content_ticker])
    if len(subtitle_sector) != 0:
      content_sector_mentions[index] = content_sector
    i += 1
    # if i == 10:
    #   break
    if i % 500 == 0:
      print("Finished adding columns to", i, "records")
print("Finished all records")
wsj['Tickers mentioned in title'] = title_ticker_mentions
wsj['Tickers mentioned in subtitle'] = subtitle_ticker_mentions
wsj['Tickers mentioned in content'] = content_ticker_mentions
wsj['Company names mentioned in title'] = title_company_mentions
wsj['Company names mentioned in subtitle'] = subtitle_company_mentions
wsj['Company names mentioned in content'] = content_company_mentions
wsj['Sectors mentioned in title'] = title_sector_mentions
wsj['Sectors mentioned in subtitle'] = subtitle_sector_mentions
wsj['Sectors mentioned in content'] = content_sector_mentions
wsj.to_csv(path+'oct20-oct21_revamped.csv', index=False)

Beginning to add columns to 15414 articles
Finished adding columns to 500 records
Finished adding columns to 1000 records
Finished adding columns to 1500 records
Finished adding columns to 2000 records
Finished adding columns to 2500 records
Finished adding columns to 3000 records
Finished adding columns to 3500 records
Finished adding columns to 4000 records
Finished adding columns to 4500 records
Finished adding columns to 5000 records
Finished adding columns to 5500 records
Finished adding columns to 6000 records
Finished adding columns to 6500 records
Finished adding columns to 7000 records
Finished adding columns to 7500 records
Finished adding columns to 8000 records
Finished adding columns to 8500 records
Finished adding columns to 9000 records
Finished adding columns to 9500 records
Finished adding columns to 10000 records
Finished adding columns to 10500 records
Finished adding columns to 11000 records
Finished adding columns to 11500 records
Finished adding columns to 12000 r